In [4]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
from sklearn.metrics import confusion_matrix

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [6]:
device = torch.device("cpu")

In [7]:
model_ft = models.efficientnet_b4()

num_ftrs = model_ft.classifier[1].in_features

model_ft.fc = nn.Linear(num_ftrs, 11)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-no_weather_class-efinetb4-bestloss-augm.pth',map_location ='cpu'))

<All keys matched successfully>

In [8]:
nb_classes = 11

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        from sklearn.metrics import classification_report

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])
target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния', 'Нет погоды' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.70      0.84      0.76        99
     облачно       0.74      0.79      0.76       129
       туман       0.95      0.78      0.86       147
       мороз       0.90      0.83      0.86       165
        град       0.87      0.91      0.89        64
      молния       0.91      1.00      0.96        43
  Нет погоды       0.18      0.01      0.02       177
       дождь       0.00      0.00      0.00        28
      радуга       0.00      0.00      0.00       162
        снег       0.00      0.00      0.00        69
      восход       0.00      0.00      0.00         0

    accuracy                           0.50      1083
   macro avg       0.48      0.47      0.46      1083
weighted avg       0.54      0.50      0.50      1083



D:\hroneniyas\anacondas\envs\gputorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\hroneniyas\anacondas\envs\gputorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\hroneniyas\anacondas\envs\gputorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        ясно       0.81      0.70      0.75       199
     облачно       0.74      0.79      0.76       258
       туман       0.91      0.88      0.90       294
       мороз       0.90      0.91      0.91       330
        град       0.98      0.96      0.97       129
      молния       0.99      0.97      0.98        86
  Нет погоды       0.95      0.96      0.95       156
       дождь       0.84      0.86      0.85       354
      радуга       0.96      0.91      0.94        57
        снег       0.78      0.83      0.80       324
      восход       1.00      0.94      0.97       139

    accuracy                           0.87      2326
   macro avg       0.90      0.88      0.89      2326
weighted avg       0.87      0.87      0.87      2326

